In [ ]:
import pandas as pd
import pickle
from scipy import stats
import pandas as pd
import seaborn as sns
import pingouin as pg
import statsmodels.formula.api as s
import numpy as np

In [ ]:
# load data
dropout = pickle.load(open( '../00_data/final/dropout_per_session.pkl','rb') )
dropout = dropout[['UserID','dropout']]
final = pickle.load(open( '../00_data/final/competence.pkl','rb') )


In [ ]:
# define high and low performer (by ability)
n = 10
final = final.sort_values(by=['ability'])
low_performer = final.head(int(len(final)*(n/100)))
final = final.sort_values(by=['ability'], ascending=False)
high_performer = final.head(int(len(final)*(n/100)))

# merge
high_performer = pd.merge(high_performer, dropout, on='UserID', how='left')
low_performer = pd.merge(low_performer, dropout, on='UserID', how='left')

In [ ]:
# split by interventiongroup
low_intvcontrol = low_performer[(low_performer['interventiongroup']=='control')]
low_intv2 = low_performer[(low_performer['interventiongroup']=='2')]
low_intv3 = low_performer[(low_performer['interventiongroup']=='3')]
low_intv4 = low_performer[(low_performer['interventiongroup']=='4')]
low_intv5 = low_performer[(low_performer['interventiongroup']=='5')]
low_intv6 = low_performer[(low_performer['interventiongroup']=='6')]

high_intvcontrol = high_performer[(high_performer['interventiongroup']=='control')]
high_intv2 = high_performer[(high_performer['interventiongroup']=='2')]
high_intv3 = high_performer[(high_performer['interventiongroup']=='3')]
high_intv4 = high_performer[(high_performer['interventiongroup']=='4')]
high_intv5 = high_performer[(high_performer['interventiongroup']=='5')]
high_intv6 = high_performer[(high_performer['interventiongroup']=='6')]

In [ ]:
# plot high performer
sns.histplot(data=high_performer, x="dropout")

In [ ]:
# plot low performer
sns.histplot(data=low_performer, x="dropout")

Effect sizes

In [ ]:
# effect sizes high performer
dropout_by_group = high_performer.groupby(['interventiongroup']).agg({'dropout':'mean','Geschlecht':'count'})
dropout_by_group = dropout_by_group.rename(columns={'Geschlecht':'count'})

In [ ]:
# effect sizes low performer
dropout_by_group = low_performer.groupby(['interventiongroup']).agg({'dropout':'mean','Geschlecht':'count'})
dropout_by_group = dropout_by_group.rename(columns={'Geschlecht':'count'})

In [ ]:
# Kruskal Wallis Test High Performer
stats.kruskal(high_intvcontrol['dropout'], high_intv2['dropout'], high_intv3['dropout'],high_intv4['dropout'],high_intv5['dropout'],high_intv6['dropout'])

In [ ]:
# Kruskal Wallis Test Low Performer
stats.kruskal(low_intvcontrol['dropout'], low_intv2['dropout'], low_intv3['dropout'],low_intv4['dropout'],low_intv5['dropout'],low_intv6['dropout'])

In [ ]:
#Wilcoxon Mann Whitney Test
print(stats.mannwhitneyu(high_intvcontrol['dropout'], high_intv2['dropout']))
print(stats.mannwhitneyu(high_intvcontrol['dropout'], high_intv3['dropout']))
print(stats.mannwhitneyu(high_intvcontrol['dropout'], high_intv4['dropout']))
print(stats.mannwhitneyu(high_intvcontrol['dropout'], high_intv5['dropout']))
print(stats.mannwhitneyu(high_intvcontrol['dropout'], high_intv6['dropout']))

In [ ]:
#Wilcoxon Mann Whitney Test
print(stats.mannwhitneyu(low_intvcontrol['dropout'], low_intv2['dropout']))
print(stats.mannwhitneyu(low_intvcontrol['dropout'], low_intv3['dropout']))
print(stats.mannwhitneyu(low_intvcontrol['dropout'], low_intv4['dropout']))
print(stats.mannwhitneyu(low_intvcontrol['dropout'], low_intv5['dropout']))
print(stats.mannwhitneyu(low_intvcontrol['dropout'], low_intv6['dropout']))